### And now - Week 3 Day 3

## AutoGen Core

Something a little different.

This is agnostic to the underlying Agent framework

You can use AutoGen AgentChat, or you can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

### The fundamental principle

Autogen Core decouples an agent's logic from how messages are delivered.  
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.

The communication infrastructure is called a Runtime.

There are 2 types: **Standalone** and **Distributed**.

Today we will use a standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.

Tomorrow we'll briefly look at a Distributed runtime.


------

Autogen core is all about decoupling logic of an agent , what it actually does, to how messages get delivered to them, the framework handels creation and communication


the autogens are responsible for their logic that is not the remit of the autogen core, it just deals with letting them play

autogen core lets them play through runtime, and there are two types of runtime, runtime is the kind of autogen core world where the autogen core works

two types of the runtime are: (1) standalone (2) distributed


In [3]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)


True

### First we define our Message object

Whatever structure we want for messages in our Agent framework.

in the langgraph we started defining the thing using the state, here we define the message instead

In [4]:
# Let's have a simple one!

@dataclass
class Message:
    content: str


# Understanding the Autogen Core Agent

The agent we are about to create in **Autogen Core** is distinct from the one we created in **Agent Chat**. Here's a breakdown of what makes the Autogen Core agent unique:

- **Wrapper Concept**: The agent in Autogen Core acts as a lightweight wrapper. It defines an entity that:
  - Can be **messaged**.
  - Can be **created**.
  - Can be **referenced**.

- **Responsibility Delegation**: Unlike a fully functional agent, the Autogen Core agent is more of a **management layer**. It’s your responsibility to delegate its tasks to other components or systems.

- **Structure**: The agent consists of two key parts:
  - **Type**: Defines the category or role of the agent.
  - **Key**: A unique identifier (agent ID) for referencing the agent.

In essence, the Autogen Core agent provides a flexible framework for managing interactions, but the actual implementation and logic depend on how you delegate its responsibilities.

----

using this every agent will be able to interact with another through the id\
agent class you make is not an LLm, agent, agent chat, its just a management holder object that has a type and a key


### Now we define our Agent

A subclass of RoutedAgent.

Every Agent has an **Agent ID** which has 2 components:  
`agent.id.type` describes the kind of agent it is  
`agent.id.key` gives it its unique identifier

Any method with the `@message_handler` decorated will have the opportunity to receive messages.


In [ ]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")
        

### OK let's create a Standalone runtime and register our agent type

In [ ]:

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
#  this is the way to register an agent to the runtime

AgentType(type='simple_agent')

### Alright! Let's start a runtime and send a message

In [7]:
runtime.start()

In [8]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well hi there!' and I disagree.


In [9]:
await runtime.stop()
await runtime.close()

### OK Now let's do something more interesting

We'll use an AgentChat Assistant!

In [ ]:
import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",  api_key = GEMINI_API_KEY)

In [16]:

class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",  api_key = GEMINI_API_KEY)
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)
    


In [ ]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [18]:
runtime.start()  # Start processing messages in the background.
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

LLMAgent received message: Hi there!
LLMAgent responded: Hi! How can I help you today?

>>> Hi! How can I help you today?

>>> This is simple_agent-default. You said 'Hi! How can I help you today?
' and I disagree.
LLMAgent received message: This is simple_agent-default. You said 'Hi! How can I help you today?
' and I disagree.
LLMAgent responded: I apologize that my initial response wasn't helpful or appropriate. As a large language model, I am still under development, and sometimes my responses can be generic or miss the mark.

To better assist you, could you tell me what kind of interaction you were expecting or hoping for? Knowing what you were looking for will help me improve.



In [20]:
await runtime.stop()
await runtime.close()

RuntimeError: Runtime is not started

### OK now let's show this at work - let's have 3 agents interact!

In [59]:
from autogen_ext.models.ollama import OllamaChatCompletionClient


class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",  api_key = GEMINI_API_KEY, temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        model_client = OpenAIChatCompletionClient(model="gemini-1.5-flash",  api_key = GEMINI_API_KEY, temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [60]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",  api_key = GEMINI_API_KEY)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)


In [61]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [62]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: rock

Player 2: rock
TERMINATE

It's a tie.
TERMINATE



In [63]:
await runtime.stop()
await runtime.close()